In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# main setting 
pd.set_option("display.max_column",None)
pd.set_option("display.min_row",90)

# Combining the DataFrames

In [3]:
app_list=pd.read_csv("app_list.csv")
steam_app_data=pd.read_csv("steam_app_data.csv")
steamspy_data=pd.read_csv("steamspy_data.csv")


In [4]:
steam_app_data.rename(columns={"steam_appid":"appid"},inplace=True)
df=app_list.merge(steam_app_data,on="appid",how="inner")
df=df.merge(steamspy_data,how="inner",on="appid")

### functions will be used within the project


In [5]:
def tostr(x):   # to change the return list into a string
    if len(x)>0:
        temp=str(x).replace("[","")
        temp=temp.replace("]","")
        temp=temp.replace("'","")
        return temp
    else:
        x="Unkown"
        return x
    
def clean_tag(x):       # to clean tag column
    if x!="Unknown":
        unclean_list=[]
        clean_list=[]
        for i in list(x.split(",")):
            unclean_list.append(i.split(":")[0])
        for i in unclean_list:
            temp=i.replace("{","")
            temp=temp.replace("'","")
            temp=temp.replace(" ","")
            clean_list.append(temp)
    result=tostr(clean_list)
    return result

def clean_achievements(x): 
    unclean_list=[]
    clean_list=[]
    if x!="Unknown":
        for i in list(x.split(",")):
            unclean_list.append(i[i.find("'name':"):-1])
        unclean_list=list(set(unclean_list))
        unclean_list
        for i in unclean_list:
            temp=i.replace("'name': '","")
            clean_list.append(temp)
        clean_list=clean_list[1:]
        result=tostr(clean_list)
        return result
    else:
        return "Unknown"
    
def cleaning_with_word_description(x):
    unclean_list=[]
    clean_list=[]
    if x!= "Unknown":
        for i in list(x.split(",")):
            unclean_list.append(i[i.find("'description'"):-1])
        for i in unclean_list:
            temp=i.replace("description': '","")
            temp=temp.replace("'","")
            temp=temp.replace("}","")
            clean_list.append(temp)
        clean_list=list(set(clean_list))[1:]
        
    result=tostr(clean_list)
    return result

# exploring the columns

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29233 entries, 0 to 29232
Data columns (total 59 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   appid                    29233 non-null  int64  
 1   name_x                   29228 non-null  object 
 2   type                     29084 non-null  object 
 3   name_y                   29232 non-null  object 
 4   required_age             29084 non-null  float64
 5   is_free                  29084 non-null  object 
 6   controller_support       5998 non-null   object 
 7   dlc                      4975 non-null   object 
 8   detailed_description     29058 non-null  object 
 9   about_the_game           29058 non-null  object 
 10  short_description        29058 non-null  object 
 11  fullgame                 0 non-null      float64
 12  supported_languages      29070 non-null  object 
 13  header_image             29084 non-null  object 
 14  website               

In [7]:
df.isna().sum() 

appid                          0
name_x                         5
type                         149
name_y                         1
required_age                 149
is_free                      149
controller_support         23235
dlc                        24258
detailed_description         175
about_the_game               175
short_description            175
fullgame                   29233
supported_languages          163
header_image                 149
website                     9983
pc_requirements              149
mac_requirements             149
linux_requirements           149
legal_notice               19167
drm_notice                 29075
ext_user_account_notice    28721
developers                   264
publishers                   149
demos                      27094
price_overview              3712
packages                    3370
package_groups               149
platforms                    149
metacritic                 26252
reviews                    23328
categories

In [8]:
df.columns

Index(['appid', 'name_x', 'type', 'name_y', 'required_age', 'is_free',
       'controller_support', 'dlc', 'detailed_description', 'about_the_game',
       'short_description', 'fullgame', 'supported_languages', 'header_image',
       'website', 'pc_requirements', 'mac_requirements', 'linux_requirements',
       'legal_notice', 'drm_notice', 'ext_user_account_notice', 'developers',
       'publishers', 'demos', 'price_overview', 'packages', 'package_groups',
       'platforms', 'metacritic', 'reviews', 'categories', 'genres',
       'screenshots', 'movies', 'recommendations', 'achievements',
       'release_date', 'support_info', 'background', 'content_descriptors',
       'name', 'developer', 'publisher', 'score_rank', 'positive', 'negative',
       'userscore', 'owners', 'average_forever', 'average_2weeks',
       'median_forever', 'median_2weeks', 'price', 'initialprice', 'discount',
       'languages', 'genre', 'ccu', 'tags'],
      dtype='object')

##### selecting the requiured columns

In [9]:
df=df[['appid', 'name_x', 'type','is_free','supported_languages','developers','publishers','platforms', 'metacritic','categories', 'genres','achievements','release_date',"genre",'tags','price_overview']]

# Cleaning and Extracting data from selected columns

In [10]:
df.fillna("Unknown",inplace=True)
df.rename(columns={"name_x":"name"},inplace=True)

In [11]:
df["release_date"]=df["release_date"].str.split(":",expand=True)[2].str.replace("}"," ")
df["release_date"]=df["release_date"].str.replace("'"," ")
df["release_date"]=df["release_date"].str.strip()
df["release_date"]=pd.to_datetime(df["release_date"],format="%d %b, %Y",errors='coerce')

In [12]:
df['price_overview'].str.split(",",expand=True)
df['price_overview'].str.split(",",expand=True)[1].str.split(":",expand=True)[1].str.strip()
initial_price=df['price_overview'].str.split(",",expand=True)[1].str.split(":",expand=True)[1].str.strip()
initial_price.str[0:-2]+ "."+initial_price.str[-2:]
df["initial_price"]=initial_price.str[0:-2]+ "."+initial_price.str[-2:]

######

final_price=df['price_overview'].str.split(",",expand=True)[2].str.split(":",expand=True)[1]
df["final_price"]=final_price.str[:-2] + "." +final_price.str[-2:]


####

df.drop(columns="price_overview",inplace=True)

In [13]:
df['supported_languages']=df['supported_languages'].str.split("<",expand=True)[0]

In [14]:
df["developers"]=df["developers"].str.replace("]"," ")
df["developers"]=df["developers"].str.replace("["," ")
df["developers"]=df["developers"].str.replace("'"," ")
df["developers"]=df["developers"].str.strip()


In [15]:
df["publishers"]=df["publishers"].str.replace("]"," ")
df["publishers"]=df["publishers"].str.replace("["," ")
df["publishers"]=df["publishers"].str.replace("'"," ")
df["publishers"]=df["publishers"].str.strip()


In [16]:
temp=df["platforms"].str.split(",",expand=True)[1]
mac_mask=temp.str.split(":",expand=True)[1]
df["OS_mac"]=mac_mask.str.strip()

temp=df["platforms"].str.split(",",expand=True)[2]
linux_mask=temp.str.split(":",expand=True)[1].str.replace("}"," ")
df["OS_linux"]=linux_mask.str.strip()

temp=df["platforms"].str.split(",",expand=True)[0]
win_mask=temp.str.split(":",expand=True)[1]
df["OS_windows"]=win_mask.str.strip()


df.drop(columns="platforms",inplace=True)

In [17]:
df["genres"]=df["genres"].apply(cleaning_with_word_description)
df["categories"]=df["categories"].apply(cleaning_with_word_description)
df["achievements"]=df["achievements"].apply(clean_achievements)
df["tags"]=df["tags"].apply(clean_tag)
df["release_date"]=pd.to_datetime(df["release_date"])

In [18]:
df.metacritic=df.metacritic.apply(lambda x : x[9:13] if x != "Unknown" else x)
# Note: i didn't use from 10:12 becasue probably there is a game with 100 score -like mario :)
df.metacritic=df.metacritic.str.strip()
df.metacritic=df.metacritic.str.replace(",","")
df.metacritic=df.metacritic.apply(lambda x : float(x) if x != "Unknown" else np.nan)

In [19]:
df.head(3)

,appid,name,type,is_free,supported_languages,developers,publishers,metacritic,categories,genres,achievements,release_date,genre,tags,initial_price,final_price,OS_mac,OS_linux,OS_windows
0,10,Counter-Strike,game,False,English,Valve,Valve,88.0,"Online Multi-Player, Local Multi-Player, Multi...",Action,Unkown,2000-11-01,Action,"Action, FPS, Multiplayer, Shooter, Classic, Te...",7.19,7.19,True,True,True
1,20,Team Fortress Classic,game,False,"English, French, German, Italian, Spanish - Sp...",Valve,Valve,NaN,"Online Multi-Player, Local Multi-Player, Multi...",Action,Unkown,1999-04-01,Action,"Action, FPS, Multiplayer, Classic, Shooter, Cl...",3.99,3.99,True,True,True
2,30,Day of Defeat,game,False,"English, French, German, Italian, Spanish - Spain",Valve,Valve,79.0,"Multi-player, Valve Anti-Cheat enabled",Action,Unkown,2003-05-01,Action,"FPS, WorldWarII, Multiplayer, Action, Shooter,...",3.99,3.99,True,True,True


In [21]:
df.to_csv("cleaned_data.csv")